In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

hps = utils.get_hparams_from_file("./configs/genshin_base_ms.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint('./checkpoints/G_2036.pth', net_g, None)
# _ = utils.load_checkpoint('../VITS_model/pretrain/G_809000.pth', net_g, None)
# _ = utils.load_checkpoint('../VITS_model/proper/G_1803.pth', net_g, None)
speaker = ['派蒙', '神子', '万叶', '纳西妲',\
            '胡桃', '绫华', '宵宫', '甘雨',\
            '莫娜', '影']

import soundfile as sf
import random
f = open('./filelists/val_filelist.txt', 'r', encoding='utf8')
lines = f.readlines()
f.close()

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


ImportError: attempted relative import with no known parent package

In [ ]:
with torch.no_grad():
    text = ''
    sid = 2
    stn_tst = get_text(text, hps)
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667,\
            noise_scale_w=0.8, length_scale=1.)[0][0,0].data.cpu().float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

In [3]:

randomIndex = random.sample(range(len(lines)), 1)[0]
text = lines[randomIndex].strip().split('\t')[-1]
thisSpeaker = speaker[int(lines[randomIndex].strip().split('\t')[1])]
speed = 1. #@param {type:"slider", min:0.1, max:3, step:0.05}
filename = 'test' #@param {type: "string"}
print(f'{thisSpeaker}: {text}')
with torch.no_grad():
    for i in range(len(speaker)):
        stn_tst = get_text(text, hps)
        x_tst = stn_tst.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
        sid = torch.LongTensor([i]).cuda()#@param {type:"longtensor", 0:9}
        audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667,\
            noise_scale_w=0.8, length_scale=speed)[0][0,0].data.cpu().float().numpy()
        print(f'{speaker[sid]}:')
        ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))        
        audio_path = f'./output/{speaker[sid]}.wav'
        sf.write(audio_path,audio,samplerate=hps.data.sampling_rate)

影: 初代的天理奉行，就是用手中的刀剑证明了他的意志，才会有传承到现在的九条加。
派蒙:


神子:


万叶:


纳西妲:


胡桃:


绫华:


宵宫:


甘雨:


莫娜:


影:


In [ ]:
import numpy as np


def draw_loss(length):
    path = './checkpoints/losslog/'
    losses = [[] for i in range(length)]
    losses_name = ['epoch', 'loss_disc', 'loss_gen', 'loss_fm', 'loss_mel', 'loss_dur', 'loss_kl', 'lr']
    with open(path + 'losslog.txt', 'r') as f:
      lines = f.readlines()
      if not lines:
        return
      for line in lines:
        line = line.strip().split('\t')
        for idx in range(len(losses)):
          losses[idx].append(float(line[idx]))
    x1 = losses[0]
    plt.title('Train loss vs. epoches', fontsize=20)
    plt.cla()
    i = 4
    print(np.array(losses).shape)
    plt.plot(x1, losses[i][:], '.-')
    plt.xlabel('epoches', fontsize=20)
    plt.ylabel(f'{losses_name[i]}', fontsize=20)
    plt.grid()
    plt.savefig(path + f"{losses_name[i]}.png")
    plt.plot()

draw_loss(8)

In [1]:
'a'
ls = ['a', 'b ', 123]
ls.index('a')

0